In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from sklearn.model_selection import GridSearchCV, cross_val_score, RandomizedSearchCV
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.decomposition import PCA


In [4]:
data = pd.read_csv("./train.csv")
X = data.iloc[:, 6:]
classifiers = data.iloc[:, 1:4]
enc = OneHotEncoder()
data_oh = enc.fit_transform(classifiers).toarray()
data_oh = pd.DataFrame(data_oh)


In [23]:
#spectrum_filtered = pd.DataFrame(savgol_filter(spectrum, 7, 3, deriv = 2, axis = 0))
#spectrum_filtered_st = zscore(spectrum_filtered, axis = 1)

y = data["PURITY"]/100
print(y)

# Appliquer PCA pour réduire la dimensionnalité
pca = PCA(n_components=100) 
X_pca = pca.fit_transform(X)


X_train, X_valid, y_train, y_valid = train_test_split(X_pca, y, test_size=0.2, random_state=42)

standardizer = StandardScaler()
X_train_standardized = standardizer.fit_transform(X_train)
X_valid_standardized = standardizer.transform(X_valid)

# Convert to tensors
X_train_tensor = torch.tensor(X_train_standardized, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).reshape(-1, 1)
X_valid_tensor = torch.tensor(X_valid_standardized, dtype=torch.float32)
y_valid_tensor = torch.tensor(y_valid.values, dtype=torch.float32).reshape(-1, 1)

0       0.585000
1       0.186000
2       0.198817
3       0.525000
4       0.489000
          ...   
1295    0.101000
1296    0.449000
1297    0.534000
1298    0.136000
1299    0.196000
Name: PURITY, Length: 1300, dtype: float64


[8.63385983e-01 1.19426741e-01 1.42997513e-02 1.98910919e-03
 4.96149909e-04 1.60475196e-04 9.86566323e-05 4.91435637e-05
 3.35702776e-05 1.86516555e-05 1.37499171e-05 8.14845972e-06
 4.42312642e-06 3.64445868e-06 2.58705888e-06 2.06030889e-06]


In [7]:
# Initialiser le modèle
input_size = X_train_tensor.shape[1]
net = NeuralNetRegressor(input_size=input_size)

# Définir les paramètres pour GridSearch
params = {
    'eta': [0.0005, 0.001],
    'max_epochs': [50],
    'fc1_out_features': [32, 64, 128],
    'fc2_out_features': [32, 64, 128],
    'batch': [65, 70],
    'dropout_rate': [0.3, 0.4, 0.5]
}

# Initialiser GridSearchCV
grid_search = GridSearchCV(net, params, refit=True, cv=5, scoring='neg_mean_squared_error', verbose=0)

# Entraîner le modèle avec GridSearch
grid_result = grid_search.fit(X_train_tensor, y_train_tensor)
nouveau_model = grid_result.best_estimator_

print("Best MSE: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

C:\Users\elvua\AppData\Local\Temp\ipykernel_6436\988970889.py:34: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_tensor = torch.tensor(X, dtype=torch.float32)
C:\Users\elvua\AppData\Local\Temp\ipykernel_6436\988970889.py:35: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_tensor = torch.tensor(y, dtype=torch.float32)
C:\Users\elvua\AppData\Local\Temp\ipykernel_6436\988970889.py:47: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_tensor = torch.tensor(X, dtype=torch.float32)
C:\Users\elvua\AppData\Local\Temp\ipykernel_6436\988970889.py:34

Best MSE: -286.016510 using {'batch': 45, 'dropout_rate': 0.5, 'eta': 0.005, 'fc1_out_features': 64, 'fc2_out_features': 128, 'max_epochs': 150}


In [21]:
# Initialiser le modèle
input_size = X_train_tensor.shape[1]
net = NeuralNetRegressor(input_size=input_size)

# Définir les paramètres pour GridSearch
params_dist = {
    'eta': loguniform(1e-4, 1e-1),
    'max_epochs': randint(50, 150),
    'fc1_out_features': randint(32, 64, 128),
    'fc2_out_features': randint(32, 64, 128),
    'batch': randint(32, 70),
    'dropout_rate': uniform(0,0.5)
}

# Initialiser RandomizedSearchCV
random_search = RandomizedSearchCV(net, params_dist, refit=True, cv=5, scoring='neg_mean_squared_error', verbose=0)

# Entraîner le modèle avec GridSearch
random_grid_result = random_search.fit(X_train_tensor, y_train_tensor)
nouveau_model = random_grid_result.best_estimator_

print("Best MSE: %f using %s" % (random_grid_result.best_score_, random_grid_result.best_params_))

/var/folders/93/xgf_b79s5sd1vjn_794f14ym0000gn/T/ipykernel_1170/3745802480.py:34: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_tensor = torch.tensor(X).clone().detach().float()
/var/folders/93/xgf_b79s5sd1vjn_794f14ym0000gn/T/ipykernel_1170/3745802480.py:35: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_tensor = torch.tensor(y).clone().detach().float()
/var/folders/93/xgf_b79s5sd1vjn_794f14ym0000gn/T/ipykernel_1170/3745802480.py:55: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_tensor = torch.tensor(X, dtype=torch.float32)
/var/fol

Best MSE: -0.003837 using {'batch': 56, 'dropout_rate': 0.14561457009902096, 'eta': 0.006847920095574782, 'fc1_out_features': 169, 'fc2_out_features': 191, 'max_epochs': 141}


In [22]:
y_pred = nouveau_model.predict(X_valid_tensor)
y_pred_train=nouveau_model.predict(X_train_tensor)

# Vérifiez les sorties du modèle
print("y_pred contains NaN:", np.isnan(y_pred).any())

# Calculer la MSE
mse = np.mean(((y_pred - y_valid)*100)**2)
print("MSE :", mse)

# Calculer le t_score
train_score = np.mean(np.abs((y_pred_train-y_train)*100<=5))
test_score = np.mean(np.abs((y_pred-y_valid)*100<=5))
print("t_score test :", train_score)
print("t_score train :", test_score)

"""
# Afficher la courbe d'apprentissage
import matplotlib.pyplot as plt
plt.plot(steps, learning_curve)
plt.xlabel('Steps')
plt.ylabel('Loss')
plt.title('Learning Curve')
plt.show()
"""

y_pred contains NaN: False
MSE : 42.40179129158316
t_score test : 0.9230769230769231
t_score train : 0.9038461538461539


/var/folders/93/xgf_b79s5sd1vjn_794f14ym0000gn/T/ipykernel_1170/3745802480.py:55: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_tensor = torch.tensor(X, dtype=torch.float32)


"\n# Afficher la courbe d'apprentissage\nimport matplotlib.pyplot as plt\nplt.plot(steps, learning_curve)\nplt.xlabel('Steps')\nplt.ylabel('Loss')\nplt.title('Learning Curve')\nplt.show()\n"